# Heavy-Tailed Distributions

# New website

We have moved this lecture to a new lecture series.

See [An Undergraduate Lecture Series for the Foundations of Computational Economics](https://intro.quantecon.org)